# Анализ данных сервиса для чтения книг

**Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта. 

**Цель исследования** — анализ данных о книгах, издательствах, авторах, пользовательских оценках книг и обзорах на книги.

**Описание данных**

Таблица `books` содержит данные о книгах:

- `book_id` — идентификатор книги;

- `author_id` — идентификатор автора;

- `title` — название книги;

- `num_pages` — количество страниц;

- `publication_date` — дата публикации книги;

- `publisher_id` — идентификатор издателя.

Таблица `authors` содержит данные об авторах:

- `author_id` — идентификатор автора;

- `author` — имя автора.

Таблица `publishers` содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;

- `publisher` — название издательства.

Таблица `ratings` содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;

- `book_id` — идентификатор книги;

- `username` — имя пользователя, оставившего оценку;

- `rating` — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;

- `book_id` — идентификатор книги;

- `username` — имя пользователя, написавшего обзор;

- `text` — текст обзора.

**План работы**

1. Исследование таблиц — вывод первых строк таблиц, подсчёт количества строк в каждой таблице.

2. Подсчёт количества книг, вышедших после 1 января 2000 года.

3. Подсчёт количества обзоров и средней оценки для каждой книги.

4. Определение издательства, которое выпустило наибольшее число книг толще 50 страниц (для исключения из анализа брошюр).

5. Выявление автора с самой высокой средней оценкой книг (с учётом только книг с 50 и более оценками).

6. Подсчёт среднего количества обзоров от пользователей, которые поставили больше 48 оценок.

7. Описание выводов по каждой из решённых задач.

## Исследование таблиц

Импортируем необходимые библиотеки и модули.

In [7]:
import pandas as pd
# импорт библиотеки pandas
import sqlalchemy as sa
# импорт библиотеки sqlalchemy

Установим параметры.

In [9]:
db_config = {
'user': 'praktikum_student', 
# имя пользователя
'pwd': 'Sdf4$2;d-d30pp', 
# пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
# хост
'port': 6432, 
# порт подключения
'db': 'data-analyst-final-project-db'
# название базы данных
}
# создание словаря ‘db_config’ с ключами и значениями
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(
    **db_config)
# создание переменной ‘connection_string’
# использование параметров из словаря ‘db_config’

Сохраним коннектор.

In [11]:
engine = sa.create_engine(
connection_string, connect_args={'sslmode':'require'})
# создание объекта ‘engine’ для подключения к базе данных, 
# указанной в переменной ‘connection_string’

Напишем функцию для выполнения SQL-запроса.

In [13]:
def get_sql_data(
query:str, engine:sa.engine.base.Engine=engine
) -> pd.DataFrame:
# определение функции ‘get_sql_data’
# принятие функцией ‘get_sql_data’ двух аргументов: 
# ‘query’ (строка с запросом к базе данных)
# и ‘engine’ (объект класса sa.engine.base.Engine)
    '''Открываем соединение, получаем данные из sql, \
    закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), 
                           con = con)
# использование функцией объекта ‘engine’ 
# для подключения к базе данных
# считывание данных из запроса 
# и преобразование в датафрейм

### Вывод первых строк таблиц

Выведем первые 5 строк таблицы `books`.

In [16]:
get_sql_data('''(SELECT * 
                 FROM books 
                 LIMIT 5)''')
# выбор всех записей из таблицы ‘books’ 
# и ограничение их количества до 5
# вызов функции ‘get_sql_data’

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем первые 5 строк таблицы `authors`.

In [18]:
get_sql_data('''(SELECT * 
                 FROM authors 
                 LIMIT 5)''')
# выбор всех записей из таблицы ‘authors’ 
# и ограничение их количества до 5
# вызов функции ‘get_sql_data’

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выведем первые 5 строк таблицы `publishers`.

In [20]:
get_sql_data('''(SELECT * 
                 FROM publishers 
                 LIMIT 5)''')
# выбор всех записей из таблицы ‘publishers’ 
# и ограничение их количества до 5
# вызов функции ‘get_sql_data’

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выведем первые 5 строк таблицы `ratings`.

In [22]:
get_sql_data('''(SELECT * 
                 FROM ratings 
                 LIMIT 5)''')
# выбор всех записей из таблицы ‘ratings’ 
# и ограничение их количества до 5
# вызов функции ‘get_sql_data’

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выведем первые 5 строк таблицы `reviews`.

In [24]:
get_sql_data('''(SELECT * 
                 FROM reviews 
                 LIMIT 5)''')
# выбор всех записей из таблицы ‘reviews’ 
# и ограничение их количества до 5
# вызов функции ‘get_sql_data’

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Подсчёт количества строк в каждой таблице

Определим, сколько строк содержится в таблице `books`.

In [27]:
get_sql_data('''(SELECT COUNT(*) AS total_rows_books
                 FROM books)''')
# подсчёт общего количества строк в таблице ‘books’
# вызов функции ‘get_sql_data’

,total_rows_books
0,1000


В таблице `books` содержится 1 тыс. строк.

Определим, сколько строк содержится в таблице `authors`.

In [30]:
get_sql_data('''(SELECT COUNT(*) AS total_rows_authors
                 FROM authors)''')
# подсчёт общего количества строк в таблице ‘authors’
# вызов функции ‘get_sql_data’

,total_rows_authors
0,636


В таблице `authors` содержится 636 строк.

Определим, сколько строк содержится в таблице `publishers`.

In [33]:
get_sql_data('''(SELECT COUNT(*) AS total_rows_publishers
                 FROM publishers)''')
# подсчёт общего количества строк в таблице ‘publishers’
# вызов функции ‘get_sql_data’

,total_rows_publishers
0,340


В таблице `publishers` содержится 340 строк.

Определим, сколько строк содержится в таблице `ratings`.

In [36]:
get_sql_data('''(SELECT COUNT(*) AS total_rows_ratings
                 FROM ratings)''')
# подсчёт общего количества строк в таблице ‘ratings’
# вызов функции ‘get_sql_data’

,total_rows_ratings
0,6456


В таблице `ratings` содержится 6 тыс. 456 строк.

Определим, сколько строк содержится в таблице `reviews`.

In [39]:
get_sql_data('''(SELECT COUNT(*) AS total_rows_reviews
                 FROM reviews)''')
# подсчёт общего количества строк в таблице ‘reviews’
# вызов функции ‘get_sql_data’

,total_rows_reviews
0,2793


В таблице `reviews` содержится 2 тыс. 793 строки.

## Подсчёт количества книг, вышедших после 1 января 2000 года

Определим самую раннюю и самую позднюю дату публикации книг.

In [43]:
get_sql_data('''(SELECT MIN(publication_date) AS min_publication_date,
                        MAX(publication_date) AS max_publication_date
                 FROM books)''')
# нахождение минимального значения 
# столбца ‘publication_date’ из таблицы ‘books’
# нахождение максимального значения 
# столбца ‘publication_date’ из таблицы ‘books’
# вызов функции ‘get_sql_data’

,min_publication_date,max_publication_date
0,1952-12-01,2020-03-31


Самая ранняя дата публикации книг — 1 декабря 1952 года, а самая поздняя — 31 марта 2020 года.

Выясним, сколько книг вышло после 1 января 2000 года.

In [46]:
get_sql_data('''(SELECT COUNT(book_id) AS total_books
                 FROM books 
                 WHERE publication_date > '2000-01-01')''')
# подсчёт количества значений
# в столбце ‘book_id’ таблицы ‘books’
# при условии, что даты 
# в столбце ‘publication_date’ позже 1 января 2000 года
# вызов функции ‘get_sql_data’

,total_books
0,819


Всего в базе данных содержится информация об одной тысяче книг, большая часть которых была опубликована после 1 января 2000 года. Со 2 января 2000 года по 31 марта 2020 года вышло 819 книг, представленных в базе данных.

## Подсчёт количества обзоров и средней оценки для каждой книги

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [50]:
get_sql_data('''(SELECT b.book_id,
                        b.title,
                        COUNT(DISTINCT(re.review_id)) AS total_reviews,
                        AVG(ra.rating) AS average_rating
                 FROM books AS b
                 LEFT JOIN reviews AS re ON b.book_id = re.book_id
                 LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
                 GROUP BY b.book_id, b.title
                 ORDER BY average_rating DESC, total_reviews DESC)''')
# группировка данных по столбцам ‘book_id’ и ‘title’ таблицы ‘books’
# выполнение агрегатных функций для каждой группы
# сортировка результатов запроса по убыванию ‘average_rating’ и ‘total_reviews’
# вывод значений столбцов ‘book_id’ и ‘title’ из таблицы ‘books’
# вывод количества уникальных значений ‘review_id’ в таблице ‘reviews’
# вывод средних значений столбца ‘rating’ из таблицы ‘ratings’
# вызов функции ‘get_sql_data’

,book_id,title,total_reviews,average_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


Определим наименьшее и наибольшее количество обзоров на книгу, а также минимальную и максимальную среднюю оценку книги.

In [52]:
get_sql_data('''(WITH
                 i AS (SELECT b.book_id,
                              b.title,
                              COUNT(DISTINCT(re.review_id)) AS total_reviews,
                              AVG(ra.rating) AS average_rating
                       FROM books AS b
                       LEFT JOIN reviews AS re ON b.book_id = re.book_id
                       LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
                       GROUP BY b.book_id, b.title
                       ORDER BY average_rating DESC, total_reviews DESC)
                 SELECT MIN(i.total_reviews) AS min_total_reviews,
                        MAX(i.total_reviews) AS max_total_reviews,
                        MIN(i.average_rating) AS min_average_rating,
                        MAX(i.average_rating) AS max_average_rating
                 FROM i)''')
# формирование подзапроса с псевдонимом i
# группировка данных по столбцам ‘book_id’ и ‘title’ таблицы ‘books’
# выполнение агрегатных функций для каждой группы
# сортировка результатов запроса по убыванию ‘average_rating’ и ‘total_reviews’
# вывод значений столбцов ‘book_id’ и ‘title’ из таблицы ‘books’
# вывод количества уникальных значений ‘review_id’ в таблице ‘reviews’
# вывод средних значений столбца ‘rating’ из таблицы ‘ratings’
# написание основного запроса
# нахождение минимального значения столбца ‘total_reviews’ из таблицы ‘i’
# нахождение максимального значения столбца ‘total_reviews’ из таблицы ‘i’
# нахождение минимального значения столбца ‘average_rating’ из таблицы ‘i’
# нахождение максимального значения столбца ‘average_rating’ из таблицы ‘i’
# вызов функции ‘get_sql_data’

,min_total_reviews,max_total_reviews,min_average_rating,max_average_rating
0,0,7,1.5,5.0


В базе данных существуют книги, на которые пользователи не написали обзоры. Наибольшее количество обзоров на одну книгу — 7.

Минимальная средняя пользовательская оценка книги составляет 1.5, а максимальная — 5.

## Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (так мы исключим из анализа брошюры).

In [56]:
get_sql_data('''(SELECT p.publisher_id,
                        p.publisher,
                        COUNT(b.book_id) AS total_books
                 FROM books AS b
                 JOIN publishers AS p ON b.publisher_id = p.publisher_id
                 WHERE b.num_pages > 50
                 GROUP BY p.publisher_id, p.publisher
                 ORDER BY total_books DESC
                 LIMIT 1)''')
# группировка данных по столбцам ‘publisher_id’ 
# и ‘publisher’ таблицы ‘publishers’
# выполнение агрегатных функций для каждой группы
# выполнение запроса только для тех записей, 
# у которых значение столбца ‘num_pages’ таблицы ‘books’ больше 50
# сортировка результатов запроса по убыванию ‘total_books’
# вывод значений столбцов ‘publisher_id’ 
# и ‘publisher’ из таблицы ‘publishers’
# вывод количества значений в столбце ‘book_id’ из таблицы ‘books’
# вывод только первой записи из результатов запроса
# вызов функции ‘get_sql_data’

,publisher_id,publisher,total_books
0,212,Penguin Books,42


Издательство «Penguin Books» выпустило наибольшее количество книг объёмом более 50 страниц — 42 книги. «Penguin Books» является британским издательством, основанным в 1935 году. Важным достижением издательства считается демократизация книжного рынка, благодаря которой книга превратилась из предмета роскоши в удовольствие, доступное для широких масс.

## Выявление автора с самой высокой средней оценкой книг

Установим, у какого автора самая высокая средняя оценка книг. Будем учитывать только книги с 50 и более оценками.

In [60]:
get_sql_data('''(WITH
                      i AS (SELECT book_id,
                                   COUNT(rating_id) AS total_ratings,
                                   ROUND(AVG(rating), 2) AS average_rating
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating_id) >= 50)
                 SELECT a.author_id,
                        a.author,
                        COUNT(i.book_id) AS total_books,
                        SUM(i.total_ratings) AS total_ratings,
                        ROUND(AVG(i.average_rating), 2) AS average_rating
                 FROM authors AS a
                 JOIN books AS b ON a.author_id = b.author_id
                 JOIN i AS i ON b.book_id = i.book_id
                 GROUP BY a.author_id, a.author
                 ORDER BY average_rating DESC
                 LIMIT 1)''')
# формирование подзапроса с псевдонимом i
# группировка данных по столбцу ‘book_id’ таблицы ‘ratings’
# выполнение агрегатных функций для каждой группы
# выполнение запроса только для тех записей, 
# где количество значений в столбце ‘rating_id’
# таблицы ‘ratings’ равно или больше 50
# вывод значений столбца ‘book_id’ из таблицы ‘ratings’
# вывод количества значений в столбце ‘rating_id’ из таблицы ‘ratings’
# вывод средних значений столбца ‘rating’ из таблицы ‘ratings’ 
# с округлением до 2-х знаков после запятой
# написание основного запроса
# группировка данных по столбцам ‘author_id’ и ‘author_id’ таблицы ‘authors’
# выполнение агрегатных функций для каждой группы
# сортировка результатов запроса по убыванию ‘average_rating’
# вывод значений столбцов ‘author_id’ и ‘author’ из таблицы ‘authors’
# вывод количества значений в столбце ‘book_id’ из таблицы ‘i’
# вывод суммы значений столбца ‘total_ratings’ из таблицы ‘i’
# вывод средних значений столбца ‘average_rating’ из таблицы ‘i’ 
# с округлением до 2-х знаков после запятой
# вывод только первой записи из результатов запроса
# вызов функции ‘get_sql_data’

,author_id,author,total_books,total_ratings,average_rating
0,236,J.K. Rowling/Mary GrandPré,4,310.0,4.29


Если рассматривать только книги с 50 и более оценками, то самая высокая средняя оценка у книг авторов Джоан Роулинг и Мэри Грандпре — 4.29. У этих авторов в базе данных представлено четыре книги. Общее количество пользовательских оценок этих книг составляет 310. 

Джоан Роулинг — британская писательница, известная как автор серии романов о Гарри Поттере. Книги о Гарри Поттере стали самой продаваемой серией книг в истории. Мэри Грандпре — американская художница и писательница, известная своими иллюстрациями к изданиям книг о Гарри Поттере.

## Подсчёт среднего количества обзоров от пользователей, которые поставили больше 48 оценок

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [64]:
get_sql_data('''(WITH 
                      i AS (SELECT username,
                                   COUNT(rating_id) AS total_ratings
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 48),
                      j AS (SELECT re.username,
                                   COUNT(re.review_id) AS total_reviews
                            FROM reviews AS re
                            JOIN i AS i ON re.username = i.username
                            WHERE re.username IN (i.username)
                            GROUP BY re.username)
                 SELECT AVG(j.total_reviews) AS average_reviews
                 FROM j)''')
# формирование первого подзапроса с псевдонимом i
# группировка данных по столбцу ‘username’ таблицы ‘ratings’
# выполнение агрегатных функций для каждой группы
# выполнение запроса только для тех записей, 
# где количество значений в столбце ‘rating_id’ таблицы ‘ratings’ больше 48
# вывод значений столбца ‘username’ из таблицы ‘ratings’
# вывод количества значений в столбце ‘rating_id’ из таблицы ‘ratings’
# формирование второго подзапроса с псевдонимом j
# группировка данных по столбцу ‘username’ таблицы ‘reviews’
# выполнение агрегатных функций для каждой группы
# выполнение запроса только для тех записей, 
# где значение столбца ‘username’ из таблицы ‘reviews’ 
# совпадает с значением столбца ‘username’ из таблицы ‘i’
# вывод значений столбца ‘username’ из таблицы ‘reviews’
# вывод количества значений в столбце ‘review_id’ из таблицы ‘reviews’
# написание основного запроса
# вывод среднего значения столбца ‘total_reviews’ из таблицы ‘j’
# вызов функции ‘get_sql_data’

,average_reviews
0,24.0


Пользователи сервиса, которые читают и оценивают много книг, также пишут довольно много отзывов на них. В среднем на пользователей, оценивших более 48 книг, приходится 24 книжных обзора.

## Выводы по результатам исследования

База данных сервиса по подписке для чтения книг включает информацию о 1000 книг, 636 авторах, 340 издательствах, 6 456 пользовательских оценках книг и 2 793 обзорах на книги.

Самая ранняя дата публикации книг в базе данных — 1 декабря 1952 года, а самая поздняя — 31 марта 2020 года. После 1 января 2000 года вышла большая часть книг, представленных в базе данных — 819 книг из 1000.

В базе данных существуют книги, на которые пользователи не написали обзоры. Наибольшее количество обзоров на одну книгу — 7. Минимальная средняя пользовательская оценка книги составляет 1.5, а максимальная — 5.

Британское издательство «Penguin Books» выпустило наибольшее количество книг объёмом более 50 страниц — 42 книги.

Если рассматривать только книги с 50 и более оценками, то самая высокая средняя оценка книг у британской писательницы Джоан Роулинг и американского иллюстратора и писательницы Мэри Грандпре — 4.29. У этих авторов в базе данных представлено четыре книги. Общее количество пользовательских оценок этих книг составляет 310.

Пользователи сервиса, которые читают и оценивают много книг, также пишут довольно много отзывов на них. В среднем на пользователей, поставивших больше 48 оценок книгам, приходится 24 книжных обзора.

Оценки и обзоры позволяют узнать предпочтения читателей, их ожидания от сервиса и книг, а также выявить возможные проблемы и недостатки. Это поможет улучшить качество приложения для любителей книг и адаптировать предложения под запросы аудитории.

Положительные отзывы и высокие оценки способствуют укреплению имиджа книжного сервиса и повышению его авторитета среди потенциальных клиентов. Это может привлечь новых пользователей и увеличить лояльность существующих.

Обзоры и оценки книг помогают пользователям сделать выбор при чтении.

Чтобы стимулировать пользователей сервиса ставить больше оценок книгам и писать больше обзоров на них, можно использовать следующие методы:
- рассказывать об авторах и книгах в социальных сетях;
- активно общаться с читателями, проводить прямые эфиры с читателями и авторами книг;
- публиковать отрывки книг в тематических сообществах;
- проводить розыгрыши книг.

На основе пользовательских оценок книг и обзоров на книги можно определить наиболее популярные и востребованные книги, а также авторов, что может повысить эффективность рекламных кампаний, ориентированных на конкретные интересы аудитории.

Изучение оценок и обзоров на книги позволяет выявить тенденции и закономерности в предпочтениях читателей, а также определить перспективные направления развития сервиса. Это поможет адаптироваться к изменениям рынка и оставаться конкурентоспособным.